## パラメータ・チューニング（グリッドサーチ）

##### サンプルデータの読み込み
ここではグリッドサーチの実行方法について学びます。<br>
まずはサンプルデータ（分類用）を読み込みます。

In [ ]:
# Breast cancer dataset for binary classification
import pandas as pd
from sklearn.datasets import load_breast_cancer

# Set X and y
dataset = load_breast_cancer()
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.Series(dataset.target, name='y')
X.join(y).head(3)

##### チューニングするパイプラインの設定
分類器としてロジスティック回帰を設定しておきます。<br>
その際、以下を認識しておきましょう。
- ここでは、学習器の略称として"est"、次元圧縮に"pca"を使います
- 本記号がグリッドサーチ時の設定にも使われます。

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

# パイプラインの設定
pipe_logistic = Pipeline([('scl', StandardScaler()),
                          ('pca', PCA(random_state=1)),
                          ('est', LogisticRegression(solver='lbfgs',random_state=1))])

##### グリッドサーチ条件の設定方法
グリッドサーチによる探索条件の設定は、以下のルールに従います。
- 上記指定の文字列"pca"や"est"と各アルゴリズムが持つパラメータ名の文字列を結合

下記設定例で、具体的に、上記ルールの設定方法を確認しましょう。
- 主成分圧縮：抽出主成分数を5,7,9の3通り
- ロジスティック回帰：正則化パラメータは0.1から100の4通り

In [ ]:
# パラメータグリッドの設定方法
param_grid_logistic = {'pca__n_components':[5,7,9],
                       'est__C':[0.1,1.0,10.0,100.0]}

##### グリッドサーチの実行（学習）
ハイパーパラメータの異なるモデルの評価はGridSerachCVで行うことができます。<br>
gsをfitした時点で、各パラメータのモデルの構築と評価を終え、ベストモデルを内部で保持するところまで全自動で動きます。

In [1]:
print('探索空間:%s' %param_grid_logistic)
gs = GridSearchCV(estimator=pipe_logistic,
                  param_grid=param_grid_logistic,
                  scoring='f1',
                  cv=3,
                  return_train_score=False)
gs.fit(X, y)

NameError: name 'param_grid_logistic' is not defined

ベストモデルで予測をしたい場合は以下です。

In [ ]:
gs.predict_proba(X)

グリッドサーチの探索結果を閲覧したい場合は以下です。

In [ ]:
pd.DataFrame(gs.cv_results_).head()

以上でグリッドサーチの実行は終了です。

<b>[確認してみよう]</b>Scikit-learn公式ドキュメントで、GridSearchCVクラスのメソッド（precidt, predict_proba）が、ベストモデルで行われているということを確認しよう。
- 以下のページで、Methods部分の説明文を読んでみよう
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV